In [42]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [43]:
import os
import pandas as pd
import numpy as np
import datetime
import pickle as pkl
from sklearn.linear_model import LinearRegression, ElasticNet
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.model_selection import learning_curve, GridSearchCV

In [74]:
from divvy.ml_logic.data_import import get_divvy_data, get_weather_data, get_station_data
from divvy.ml_logic.cleaning import cleaning_divvy_gen,cleaning_divvy_gen_agg,weather_cleaning
from divvy.ml_logic.cleaning import features_target, merge_divvy_weather, compute_geohash_stations
from divvy.ml_logic.preprocessor import preprocess_features, target_process
from divvy.ml_logic.main import preprocess_test, preprocess
from divvy.interface_ui.flow.flow import transform_user_inputs,get_station_availability
from divvy.interface_ui.flow.ui_utils import process_weather_inputs,get_coordinates

In [75]:
departure_date=datetime.datetime(2022,12,16)
departure_time=datetime.datetime(12,10,20)
departure_address='3424 S giles'
num_nearest_stations=3

In [76]:
latlonuser, nclose_stations=transform_user_inputs(departure_address,
                                       num_nearest_stations)
nclose_stations

,Unnamed: 0,external_id,station_id,short_name,eightd_has_key_dispenser,legacy_id,electric_bike_surcharge_waiver,capacity,has_kiosk,rental_methods,...,station_type,eightd_station_services,region_id,region_code,address,dockless_bikes_parking_zone_capacity,rack_model,client_station_id,target_bike_capacity,target_scooter_capacity
98,98,a3aae35a-a135-11e9-9cda-0a87ae2ba916,a3aae35a-a135-11e9-9cda-0a87ae2ba916,13345,False,335,False,15,True,"['KEY', 'CREDITCARD', 'TRANSITCARD']",...,classic,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
553,553,a3a639f5-a135-11e9-9cda-0a87ae2ba916,a3a639f5-a135-11e9-9cda-0a87ae2ba916,13217,False,149,False,11,True,"['KEY', 'CREDITCARD', 'TRANSITCARD']",...,classic,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,7,a3a75d8a-a135-11e9-9cda-0a87ae2ba916,a3a75d8a-a135-11e9-9cda-0a87ae2ba916,TA1307000129,False,184,False,19,True,"['KEY', 'CREDITCARD', 'TRANSITCARD']",...,classic,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [77]:
latlonuser

[41.8321485, -87.61960959999999]

In [78]:
departure_date=datetime.date(2022,12,14)
departure_time=datetime.time(12,00,00)

In [79]:
weather_inputs=process_weather_inputs(departure_date,
                                          departure_time)

/Users/mariofernandez/code/G-Dolle/DIVVY_BIKE/divvy/interface_ui/flow/ui_utils.py:154: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_reduc["time_diff"] = df_reduc["user_input"] - df_reduc["hourly_data"]
/Users/mariofernandez/code/G-Dolle/DIVVY_BIKE/divvy/interface_ui/flow/ui_utils.py:158: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_data.drop(columns=["user_input","date_input","date_weather","time_diff"], inplace=True)


In [80]:
weather_inputs

,dt_iso,temp,pressure,humidity,wind_speed,wind_deg,clouds_all,hourly_data
8,2022-12-14 09:00:00,3.82,1013,85,9.82,114,100,2022-12-14 09:00:00


In [70]:
weather_inputs.shape

(1, 8)

In [85]:
sta_reduced=compute_geohash_stations()
sta_reduced.geohash.value_counts()

/Users/mariofernandez/code/G-Dolle/DIVVY_BIKE/divvy/ml_logic/cleaning.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_stations_reduced.rename(columns={"name":"station_name"}, inplace=True)


dp3wm    104
dp3wj     65
dp3wk     64
dp3wt     63
dp3ws     56
dp3tm     55
dp3w7     54
dp3we     54
dp3tw     53
dp3tq     51
dp3ty     50
dp3w5     50
dp3wu     49
dp3tt     46
dp3wh     44
dp3tv     41
dp3tu     40
dp3ts     38
dp3tr     35
dp3w6     34
dp3th     30
dp3wd     30
dp3tj     29
dp3tx     27
dp3wq     27
dp3wn     25
dp3xh     25
dp3tn     23
dp3tg     20
dp3wv     19
dp3w9     17
dp3te     15
dp3v2     13
dp3tk     13
dp3v0     11
dp3sz     10
dp3sy      9
dp3tp      8
dp3w4      7
dp3wg      4
dp3xk      4
dp3xj      4
dp3w3      2
dp3x7      1
dp3tz      1
dp3t7      1
dp3ub      1
Name: geohash, dtype: int64

In [11]:
import pickle as pkl
loaded_pp=pkl.load(open('preprocessors/preprocessor_dep.pickle','rb'))

In [40]:
loaded_pp

ColumnTransformer(n_jobs=-1,
                  transformers=[('time_preproc',
                                 Pipeline(steps=[('functiontransformer',
                                                  FunctionTransformer(func=<function transform_time_features at 0x16a9e2cb0>)),
                                                 ('columntransformer',
                                                  ColumnTransformer(remainder='passthrough',
                                                                    transformers=[('onehotencoder',
                                                                                   OneHotEncoder(categories={0: array([0, 1, 2, 3, 4, 5, 6]),
                                                                                                             1: array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12])},
                                                                                                 handle_unknown='ignore',
                                                                                                 sparse=False),
                                                                                   [2,
                                                                                    3])]))]),
                                 ['hourly_data']),
                                ('weather_scaler',
                                 Pipeline(steps=[('standardscaler',
                                                  StandardScaler())]),
                                 ['temp', 'pressure', 'humidity', 'wind_speed',
                                  'wind_deg', 'clouds_all'])])

In [17]:
X_new_processed=loaded_pp.transform(weather_inputs)

In [18]:
X_new_processed.shape

(1, 27)

In [19]:
loaded_model=pkl.load(open('models/elasticnet_departures.pickle','rb'))

In [20]:
loaded_model.predict(X_new_processed)

array([2.13587051])

In [21]:
from datetime import datetime
a='2022-12-09 12:10:20'
b,c=a.split (sep=' ')
b,c

('2022-12-09', '12:10:20')

In [22]:
date_object = datetime.strptime(b, '%Y-%m-%d').date()
date_object

datetime.date(2022, 12, 9)

In [23]:
time_object=datetime.strptime(c, '%H:%M:%S').time()
time_object

datetime.time(12, 10, 20)

In [24]:
departures=6.562701150956581
arrivals=5.49496315877833

In [26]:
stations=get_station_availability(nclose_stations,arrivals,departures)
stations

,Unnamed: 0,external_id,station_id,short_name,eightd_has_key_dispenser,legacy_id,electric_bike_surcharge_waiver,capacity,has_kiosk,rental_methods,...,eightd_station_services,region_id,region_code,address,dockless_bikes_parking_zone_capacity,rack_model,client_station_id,target_bike_capacity,target_scooter_capacity,availability
98,98,a3aae35a-a135-11e9-9cda-0a87ae2ba916,a3aae35a-a135-11e9-9cda-0a87ae2ba916,13345,False,335,False,15,True,"['KEY', 'CREDITCARD', 'TRANSITCARD']",...,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
553,553,a3a639f5-a135-11e9-9cda-0a87ae2ba916,a3a639f5-a135-11e9-9cda-0a87ae2ba916,13217,False,149,False,11,True,"['KEY', 'CREDITCARD', 'TRANSITCARD']",...,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
7,7,a3a75d8a-a135-11e9-9cda-0a87ae2ba916,a3a75d8a-a135-11e9-9cda-0a87ae2ba916,TA1307000129,False,184,False,19,True,"['KEY', 'CREDITCARD', 'TRANSITCARD']",...,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [35]:
dict2=stations.to_dict('list')
dict2

{'Unnamed: 0': [98, 553, 7],
 'external_id': ['a3aae35a-a135-11e9-9cda-0a87ae2ba916',
  'a3a639f5-a135-11e9-9cda-0a87ae2ba916',
  'a3a75d8a-a135-11e9-9cda-0a87ae2ba916'],
 'station_id': ['a3aae35a-a135-11e9-9cda-0a87ae2ba916',
  'a3a639f5-a135-11e9-9cda-0a87ae2ba916',
  'a3a75d8a-a135-11e9-9cda-0a87ae2ba916'],
 'short_name': ['13345', '13217', 'TA1307000129'],
 'eightd_has_key_dispenser': [False, False, False],
 'legacy_id': ['335', '149', '184'],
 'electric_bike_surcharge_waiver': [False, False, False],
 'capacity': [15, 11, 19],
 'has_kiosk': [True, True, True],
 'rental_methods': ["['KEY', 'CREDITCARD', 'TRANSITCARD']",
  "['KEY', 'CREDITCARD', 'TRANSITCARD']",
  "['KEY', 'CREDITCARD', 'TRANSITCARD']"],
 'name': ['Calumet Ave & 35th St',
  'Calumet Ave & 33rd St',
  'State St & 35th St'],
 'lon': [-87.618034, -87.61793, -87.62679755687714],
 'rental_uris': ["{'ios': 'https://chi.lft.to/lastmile_qr_scan', 'android': 'https://chi.lft.to/lastmile_qr_scan'}",
  "{'ios': 'https://chi.lft

In [31]:
stations_simplified_df=stations.drop(columns=['Unnamed: 0','external_id','eightd_has_key_dispenser','electric_bike_surcharge_waiver','has_kiosk','rental_methods','rental_uris','eightd_station_services','region_id','region_code','address','dockless_bikes_parking_zone_capacity','rack_model','client_station_id','target_bike_capacity','target_scooter_capacity'])

In [32]:
dictionary=stations_simplified_df.to_dict('list')

In [33]:
dictionary

{'station_id': ['a3aae35a-a135-11e9-9cda-0a87ae2ba916',
  'a3a639f5-a135-11e9-9cda-0a87ae2ba916',
  'a3a75d8a-a135-11e9-9cda-0a87ae2ba916'],
 'short_name': ['13345', '13217', 'TA1307000129'],
 'legacy_id': ['335', '149', '184'],
 'capacity': [15, 11, 19],
 'name': ['Calumet Ave & 35th St',
  'Calumet Ave & 33rd St',
  'State St & 35th St'],
 'lon': [-87.618034, -87.61793, -87.62679755687714],
 'lat': [41.831379, 41.8349, 41.8310363140161],
 'station_type': ['classic', 'classic', 'classic'],
 'availability': [1, 1, 1]}

In [36]:
sta=pd.DataFrame.from_dict(dictionary)

In [37]:
sta

,station_id,short_name,legacy_id,capacity,name,lon,lat,station_type,availability
0,a3aae35a-a135-11e9-9cda-0a87ae2ba916,13345,335,15,Calumet Ave & 35th St,-87.618034,41.831379,classic,1
1,a3a639f5-a135-11e9-9cda-0a87ae2ba916,13217,149,11,Calumet Ave & 33rd St,-87.617930,41.834900,classic,1
2,a3a75d8a-a135-11e9-9cda-0a87ae2ba916,TA1307000129,184,19,State St & 35th St,-87.626798,41.831036,classic,1


In [38]:
sta.to_csv('interface_ui/stat.csv')

[autoreload of interface_ui.flow.flow failed: Traceback (most recent call last):
  File "/Users/mariofernandez/.pyenv/versions/DIVVY_BIKE/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 261, in check
    superreload(m, reload, self.old_objects)
  File "/Users/mariofernandez/.pyenv/versions/DIVVY_BIKE/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 459, in superreload
    module = reload(module)
  File "/Users/mariofernandez/.pyenv/versions/3.10.6/lib/python3.10/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 619, in _exec
  File "<frozen importlib._bootstrap_external>", line 883, in exec_module
  File "<frozen importlib._bootstrap>", line 241, in _call_with_frames_removed
  File "/Users/mariofernandez/code/G-Dolle/DIVVY_BIKE/interface_ui/flow/flow.py", line 1, in <module>
    from flow.ui_utils import get_coordinates
ModuleNotFoundError: No module named 'flow'
]


In [41]:
docker_image=#--platform=linux/amd64 tensorflow/tensorflow:2.10.0

SyntaxError: invalid syntax (2026562256.py, line 1)